# Tensor and Add Operation

ttnn.Tensor is the central type of ttnn.

It is similar to torch.Tensor in the sense that it represents multi-dimensional matrix containing elements of a single data type.

The are a few key differences:

- ttnn.Tensor can be stored in the SRAM or DRAM of Tenstorrent devices
- ttnn.Tensor doesn't have a concept of the strides, however it has a concept of row-major and tile layout
- ttnn.Tensor has support for data types not supported by torch such as `bfp8` for example
- ttnn.Tensor's shape stores the padding added to the tensor due to TILE_LAYOUT

## Creating a tensor

The recommended way to create a tensor is by using torch create function and then simply calling `ttnn.from_torch`. So, let's import both `torch` and `ttnn`

In [1]:
import torch
import ttnn

2025-06-26 04:07:07.973 | DEBUG    | ttnn:<module>:83 - Initial ttnn.CONFIG:
Config{cache_path=/home/maxim_artemov/.cache/ttnn,model_cache_path=/home/maxim_artemov/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_should_raise_exception=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}


In [2]:
import os


And now let's create a torch Tensor and convert it to ttnn Tensor

In [3]:
ttnn_tensor = ttnn.rand((3, 4))

print(f"shape: {ttnn_tensor.shape}")
print(f"layout: {ttnn_tensor.layout}")
print(f"dtype: {ttnn_tensor.dtype}")

shape: Shape([3, 4])
layout: Layout.TILE
dtype: DataType.BFLOAT16


As expected we get a tensor of shape [3, 4] in row-major layout with a data type of float32.

## Host Storage: Borrowed vs Owned

In this particular case, ttnn Tensor will borrow the data of the torch Tensor because ttnn Tensor is in row-major layout, torch tensor is contiguous and their data type matches.

Let's print the current ttnn tensor, set element of torch tensor to 1234 and print the ttnn Tensor again to see borrowed storage in action

In [17]:
ttnn_tensor = ttnn.rand((3, 4), dtype=ttnn.float32)
torch_tensor = ttnn.to_torch(ttnn_tensor, dtype=torch.float32)
borrowed_ttnn_tensor = ttnn.from_torch(torch_tensor)
print(f"Original values:\n{borrowed_ttnn_tensor}")
torch_tensor[:] = 1234
print(f"New values are all going to be 1234:\n{borrowed_ttnn_tensor}")

Original values:
ttnn.Tensor([[ 0.87337,  0.57932,  0.70375,  0.55358],
             [ 0.96836,  0.90015,  0.82169,  0.67716],
             [ 0.77847,  0.00929,  0.08590,  0.63671]], shape=Shape([3, 4]), dtype=DataType::FLOAT32, layout=Layout::ROW_MAJOR)
New values are all going to be 1234:
ttnn.Tensor([[1234.00000, 1234.00000, 1234.00000, 1234.00000],
             [1234.00000, 1234.00000, 1234.00000, 1234.00000],
             [1234.00000, 1234.00000, 1234.00000, 1234.00000]], shape=Shape([3, 4]), dtype=DataType::FLOAT32, layout=Layout::ROW_MAJOR)


We try our best to use borrowed storage but if the torch data type is not supported in ttnn, then we don't have a choice but to automatically pick a different data type and copy data

In [5]:
torch_tensor = ttnn.to_torch(ttnn.rand((3, 4)))
ttnn_tensor = ttnn.from_torch(torch_tensor)
print("torch_tensor.dtype:", torch_tensor.dtype)
print("ttnn_tensor.dtype:", ttnn_tensor.dtype)

print(f"Original values:\n{ttnn_tensor}")
torch_tensor[0, 0] = 1234
#print(f"Original values again because the tensor doesn't use borrowed storage:\n{ttnn_tensor}")

torch_tensor.dtype: torch.bfloat16
ttnn_tensor.dtype: DataType.BFLOAT16
Original values:
ttnn.Tensor([[ 0.46094,  0.08301,  0.58984,  0.55859],
             [ 0.12988,  0.55078,  0.22363,  0.16797],
             [ 0.53125,  0.57031,  0.03247,  0.77344]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)


## Data Type

The data type of the ttnn tensor can be controlled explicitly when conversion from torch.

In [6]:
torch_tensor = ttnn.to_torch(ttnn.rand((3, 4), dtype=ttnn.float32))
ttnn_tensor = ttnn.from_torch(torch_tensor, dtype=ttnn.bfloat16)
print(f"torch_tensor.dtype: {torch_tensor.dtype}")
print(f"ttnn_tensor.dtype: {ttnn_tensor.dtype}")

torch_tensor.dtype: torch.float32
ttnn_tensor.dtype: DataType.BFLOAT16


## Layout

Tenstorrent hardware is most efficiently utilized when running tensors using [tile layout](https://tenstorrent.github.io/ttnn/latest/ttnn/tensor.html#layout).
The current tile size is hard-coded to [32, 32]. It was determined to be the optimal size for a tile given the compute, memory and data transfer constraints.


ttnn provides easy and intuitive way to convert from row-major layout to tile layout and back.

In [7]:
ttnn_tensor = ttnn.rand((3, 4), layout=ttnn.ROW_MAJOR_LAYOUT)
print(f"Tensor in row-major layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")
ttnn_tensor = ttnn.to_layout(ttnn_tensor, ttnn.TILE_LAYOUT)
print(f"Tensor in tile layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")
ttnn_tensor = ttnn.to_layout(ttnn_tensor, ttnn.ROW_MAJOR_LAYOUT)
print(f"Tensor back in row-major layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")

Tensor in row-major layout:
Shape Shape([3, 4])
Layout: Layout.ROW_MAJOR
ttnn.Tensor([[ 0.78125,  0.73828,  0.18359,  0.68750],
             [ 0.87891,  0.50000,  0.29688,  0.76562],
             [ 0.72656,  0.64844,  0.41797,  0.37891]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)
Tensor in tile layout:
Shape Shape([3, 4])
Layout: Layout.TILE
ttnn.Tensor([[ 0.78125,  0.73828,  0.18359,  0.68750],
             [ 0.87891,  0.50000,  0.29688,  0.76562],
             [ 0.72656,  0.64844,  0.41797,  0.37891]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::TILE)
Tensor back in row-major layout:
Shape Shape([3, 4])
Layout: Layout.ROW_MAJOR
ttnn.Tensor([[ 0.78125,  0.73828,  0.18359,  0.68750],
             [ 0.87891,  0.50000,  0.29688,  0.76562],
             [ 0.72656,  0.64844,  0.41797,  0.37891]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)


Note that padding is automatically inserted to put the tensor into tile layout and it automatically removed after the tensor is converted back to row-major layout

The conversion to tile layout can be done when caling `ttnn.from_torch`

In [8]:
torch_tensor = ttnn.to_torch(ttnn.rand((3, 4)))
ttnn_tensor = ttnn.from_torch(torch_tensor)
print(f"Tensor in row-major layout:\nShape {ttnn_tensor.shape}; Layout: {ttnn_tensor.layout}")

Tensor in row-major layout:
Shape Shape([3, 4]); Layout: Layout.ROW_MAJOR


Note that `ttnn.to_torch` will always convert to row-major layout

## Device storage

Finally, in order to actually utilize the tensor, we need to put it on the device. So, that we can run `ttnn` operations on it

## Open the device

Use `ttnn.open` to get a handle to the device

In [9]:
device_id = 0
device = ttnn.open_device(device_id=device_id)

[2025-06-26 04:07:08.438] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-26 04:07:08.454] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-26 04:07:08.462] [info] [tt_cluster.cpp:177] [Device] Opening user mode device driver
[2025-06-26 04:07:08.462] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-26 04:07:08.463] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-26 04:07:08.467] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-26 04:07:08.468] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-26 04:07:08.472] [info] [cluster.cpp:277] [SiliconDriver] Harvesting mask for chip 0 is 0x40 (physical layout: 0x40, logical: 0x40, si

## Initialize tensors a and b with random values using torch

Create a tensor that can be used by a `ttnn` operation with the `ttnn.TILE_LAYOUT` and put the tensor on the `device`

In [10]:
input_tensor_a = ttnn.rand((32, 32), dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)
input_tensor_b = ttnn.rand((32, 32), dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)

## Add tensor a and b

`ttnn` supports operator overloading, therefore operator `+` can be used instead of `torch.add`

In [11]:
output_tensor = input_tensor_a + input_tensor_b

## Inspect the output tensor of the add in ttnn

As can be seen the tensor of the same shape, layout and dtype is produced

In [12]:
print(f"shape: {output_tensor.shape}")
print(f"dtype: {output_tensor.dtype}")
print(f"layout: {output_tensor.layout}")

shape: Shape([32, 32])
dtype: DataType.BFLOAT16
layout: Layout.TILE


In general we expect layout and dtype to stay the same when running most operations unless explicit arguments to modify them are passed in. However, there are obvious exceptions like an embedding operation that takes in `ttnn.uint32` and produces `ttnn.bfloat16`

## Convert to torch and inspect the attributes of the torch tensor

When converting the tensor to torch, `ttnn.to_torch` will move the tensor from the device, convert to tile layout and figure out the best data type to use on the torch side

In [13]:
output_tensor = ttnn.to_torch(output_tensor)
print(f"shape: {output_tensor.shape}")
print(f"dtype: {output_tensor.dtype}")

shape: torch.Size([32, 32])
dtype: torch.bfloat16


## Close the device

Close the handle the device. This is a very important step as the device can hang currently if not closed properly

In [14]:
ttnn.close_device(device)

[2025-06-26 04:07:10.224] [info] [device.cpp:1093] [Metal] Closing device 0
[2025-06-26 04:07:10.224] [info] [device.cpp:1440] [Metal] Disabling and clearing program cache on device 0
